In [ ]:
# Import our dependencies

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.feature_selection import chi2, f_classif, SelectFromModel, SelectKBest 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

# Functions

In [ ]:
def load_dataset():
    """ Reads dataset csv and returns pandas dataframe """
    
    filepath = "../Resources/charity_data.csv"

    df = pd.read_csv(filepath, encoding="utf-8", low_memory=False)
    
    return df

In [ ]:
def clean_dataset(a_df):
    """ Returns deduped, na-dropped, useless column dropped, index-reset dataframe """    
    
    a_df = a_df.drop_duplicates()   
        
    a_df = a_df.dropna()
    
    a_df = a_df.drop(columns=["EIN", "NAME"])
        
    a_df = a_df.reset_index(drop=True)
    
    return a_df   

In [ ]:
def examine_dataset(a_df):
    """ Provides summary info and visualizations of dataset """
    
    a_df.info()
    
    print("\n\n")
                  
    # Determine the number of unique values in each column.

    for col in a_df.columns:        
        if (a_df[col].nunique() > 10):
            print(f"{col}\n\n{a_df[col].value_counts()}\n\n") 

In [ ]:
def reduce_cats(a_col, a_cutoff):
    """ Inputs are a series and a cutoff value for 'Other' """
    
    print(f"BEFORE: \n\n{df[a_col].value_counts()}\n\n")

    types_to_replace = (df[a_col].value_counts().loc[lambda x: x < int(a_cutoff)]).keys().tolist()

    for code in types_to_replace:        
        df[a_col] = df[a_col].replace(code, "Other")

    # Check to make sure binning was successful

    print(f"AFTER: \n\n{df[a_col].value_counts()}\n\n") 

In [ ]:
def encode_df(a_df):
    """ Returns one-hot encoded dataframe """
    
    categorical_list = a_df.dtypes[a_df.dtypes == "object"].index.tolist() 
    
    print(f"CATEGORIES FOR EACH CATEGORICAL FEATURE ENCODED:\n\n{a_df[categorical_list].nunique()}\n\n")
    
    concat_list = []
    
    for categorical in categorical_list:
        
        concat_list.append(pd.get_dummies(a_df[categorical], prefix=categorical, prefix_sep='_'))        
    
    concat_list.append(a_df["IS_SUCCESSFUL"])

    return pd.concat(concat_list, axis=1)  

In [ ]:
def pre_process(a_df):
    """ Make X,y ... train_test_split ... scale, fit and transform """
    
    # Split our preprocessed data into our features and target arrays

    y = enc_df["IS_SUCCESSFUL"]
    X = enc_df.drop(["IS_SUCCESSFUL"], axis=1)

    # Split the preprocessed data into a training and testing dataset

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42) 
    
    # Create a StandardScaler instances
    
    scaler = StandardScaler()

    # Fit the StandardScaler

    X_scaler = scaler.fit(X_train)

    # Scale the data

    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    print(f"TRAIN SCALED SHAPE: {X_train_scaled.shape}")
    print(f"TEST SCALED SHAPE: {X_test_scaled.shape}")

    input_dim = X_train_scaled.shape[1]  
    
    return input_dim, X_train_scaled, X_test_scaled, y_train, y_test 

In [ ]:
def make_nn(input_dim=43, num_layers=2, num_units=100, num_epochs=10):
    """ Makes sequential nn, compiles, fits, saves, and reports on loss and accuracy """    
    
    nn = tf.keras.models.Sequential()

    # First layer 
    
    nn.add(tf.keras.layers.Dense(units=num_units, input_dim=input_dim, activation="relu"))
    
    # Hidden layers
    
    for layer in range(1, num_layers):
        
        nn.add(tf.keras.layers.Dense(units=num_units, activation="relu"))

    # Output layer

    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    
    nn.summary()
    
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 
    
    fit_model = nn.fit(X_train_scaled, y_train, epochs=num_epochs)   
    
    model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
    
    print(f"\n\nLoss: {model_loss}, Accuracy: {model_accuracy}")    

    nn.save("../Models/AlphabetSoupCharity_Optimization.h5")

# Call functions

In [ ]:
# Make dataframe and examine

df = clean_dataset(load_dataset())

examine_dataset(df)

In [ ]:
# Bin columns with > 10 unique values

reduce_cats("APPLICATION_TYPE", 500)

reduce_cats("ASK_AMT", 25_000)

reduce_cats("CLASSIFICATION", 1800)

In [ ]:
df.head()

In [ ]:
enc_df = encode_df(df)

enc_df

In [ ]:
# Preprocess dataframe

input_dim, X_train_scaled, X_test_scaled, y_train, y_test = pre_process(enc_df)

In [ ]:
make_nn()

# Tinkerings

In [ ]:
best_features = SelectKBest(score_func=chi2, k=5)
fit = best_features.fit(X, y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

feature_Scores = pd.concat([dfcolumns, dfscores], axis=1)
feature_Scores.columns = ["Specs", "Score"]

print(feature_Scores.nlargest(5, "Score"))

In [ ]:
model = ExtraTreesClassifier()
model.fit(X, y)

print("\n\n", model.feature_importances_)

feat_importances = pd.Series(model.feature_importances_, index=X.columns)

feat_importances.nlargest(5).plot(kind="barh")

plt.show()

In [ ]:
corrmat = df.corr()

top_corr_features = corrmat.index

plt.figure(figsize=(20,20))

sns.heatmap(df[top_corr_features].corr(), annot=False, cmap="RdYlGn")

In [ ]:
# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
# Create a StandardScaler instances

scaler = StandardScaler()

# Fit the StandardScaler

X_scaler = scaler.fit(X_train)

# Scale the data

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

print(X_train_scaled.shape)
print(X_test_scaled.shape)

In [ ]:
clf = RandomForestClassifier(random_state=0, n_estimators=128).fit(X_train_scaled, y_train)

print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

In [ ]:
feature_importances = clf.feature_importances_ 

features = sorted(zip(df.columns, clf.feature_importances_), key = lambda x: x[1])
cols = [f[0] for f in features]
width = [f[1] for f in features]

fig, ax = plt.subplots()

fig.set_size_inches(10,200)
plt.margins(y=0.001)

ax.barh(y=cols, width=width)

plt.show()

In [ ]:
sel = SelectFromModel(clf)
sel.fit(X_train_scaled, y_train)

X_selected_train, X_selected_test, y_train, y_test = train_test_split(sel.transform(X), y, random_state=1)
scaler = StandardScaler().fit(X_selected_train)
X_selected_train_scaled = scaler.transform(X_selected_train)
X_selected_test_scaled = scaler.transform(X_selected_test)

clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_selected_train_scaled, y_train)
print(f'Training Score: {clf.score(X_selected_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_selected_test_scaled, y_test)}')